In [5]:
import os
from dotenv import load_dotenv
import numpy as np
import openai
import pandas as pd
import pickle
import tiktoken
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
    wait_random
)  # for exponential backoff

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

COMPLETIONS_MODEL = "gpt-3.5-turbo"
EMBEDDING_MODEL = "text-embedding-ada-002"

In [6]:
prompt = "What is the medicine for burning eyes?"

openai.ChatCompletion.create(
    model=COMPLETIONS_MODEL,
    messages=[{'role': 'user', 'content': prompt}]
)["choices"][0]["message"]["content"].strip(" \n")

"There are different types of medication for burning eyes depending on the cause of the problem. Some possible treatments include:\n\n1. Lubricating eye drops or artificial tears to relieve dryness or irritation caused by environmental factors like smoke, dust, or wind.\n\n2. Anti-inflammatory eye drops or ointments for redness, swelling, or infection caused by allergies, conjunctivitis (pink eye), or other eye infections.\n\n3. Prescription drugs like steroids or immune modulators for severe or chronic eye conditions like uveitis, scleritis, or keratitis.\n\nIt's important to consult an eye doctor for a proper diagnosis and treatment plan, as some eye conditions can be serious and require specialized care or surgery."

In [8]:
prompt = """Answer the question as truthfully as possible, and if you're unsure of the answer, say "Sorry, I don't know".

Q: What is the medicine for burning eyes?
A:"""

openai.ChatCompletion.create(
    model=COMPLETIONS_MODEL,
    messages=[{'role': 'user', 'content': prompt}]
)["choices"][0]["message"]["content"].strip(" \n")

"Sorry, I don't know. Burning eyes can have various causes, and the appropriate medicine would depend on the underlying condition. It's best to consult with an eye doctor or healthcare provider for a proper diagnosis and treatment."

In [9]:
# We have hosted the processed dataset, so you can download it directly without having to recreate it.
# This dataset has already been split into sections, one row for each section of the Wikipedia page.

df = pd.read_csv('clarke_symptoms_cleaned.csv')
# df = pd.read_csv('clarke_remedy_info_cleaned_v2.csv')

df = df.set_index(["title", "heading"])
print(f"{len(df)} rows in the data.")
df.sample(5)

14958 rows in the data.


,,content,tokens
title,heading,,
Physostigma,Head,"Vertigo: on moving, with fainting and muscÃ¦ v...",455
Quassia,Mind,"Awoke with great anxiety 1 a.m., unable to sle...",29
Cancer Fluviatilis,Stomach,Fulness and pressure in stomach; burning in ep...,34
Cuprum Sulphuricum,Head,"Bald spot over r. parietal bone, not depending...",18
Spongia Tosta,Respiratory Organs,"[Affections in general of larynx and trachea, ...",465


In [10]:
@retry(wait=wait_random_exponential(min=1, max=10), stop=stop_after_attempt(10))
# @retry(wait=wait_random(min=60, max=120), stop=stop_after_attempt(10))
def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> list[float]:
    result = openai.Embedding.create(
      model=model,
      input=text
    )
    return result["data"][0]["embedding"]

def compute_doc_embeddings(df: pd.DataFrame) -> dict[tuple[str, str], list[float]]:
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.
    
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_embedding(r.content) for idx, r in df.iterrows()
    }

In [11]:
def load_embeddings(fname: str) -> dict[tuple[str, str], list[float]]:
    """
    Read the document embeddings and their keys from a CSV.
    
    fname is the path to a CSV with exactly these named columns: 
        "title", "heading", "0", "1", ... up to the length of the embedding vectors.
    """
    
    df = pd.read_csv(fname, header=0)
    max_dim = max([int(c) for c in df.columns if c != "title" and c != "heading"])
    return {
           (r.title, r.heading): [r[str(i)] for i in range(max_dim + 1)] for _, r in df.iterrows()
    }

Again, we have hosted the embeddings for you so you don't have to re-calculate them from scratch.

In [12]:
# document_embeddings = load_embeddings("https://cdn.openai.com/API/examples/data/olympics_sections_document_embeddings.csv")
document_embeddings = load_embeddings("clarke_symptoms_cleaned_embeddings_v2.csv")
# document_embeddings = load_embeddings("clarke_remedy_info_cleaned_v2_embeddings_v2.csv")
# document_embeddings = load_embeddings("clarke_symptoms_cleaned_v2_nose_embeddings_v2.csv")

# ===== OR, uncomment the below line to recaculate the embeddings from scratch. ========

# document_embeddings = compute_doc_embeddings(df)

In [66]:
# pd.DataFrame(document_embeddings.items()).to_csv('clarke_remedy_info_cleaned_v2_embeddings.csv', index=False)
# pd.DataFrame(document_embeddings.items()).to_csv('clarke_symptoms_cleaned_v2_nose_embeddings.csv', index=False)

In [82]:
# An example embedding:
example_entry = list(document_embeddings.items())[0]
print(f"{example_entry[0]} : {example_entry[1][:5]}... ({len(example_entry[1])} entries)")

('Abies Canadensis', 'Mind') : [-0.000476399, 0.004719749, 0.01499975, -0.018661214, -0.015802821]... (1536 entries)


In [14]:
def vector_similarity(x: list[float], y: list[float]) -> float:
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query: str, contexts: dict[(str, str), np.array]) -> list[(float, (str, str))]:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

In [15]:
order_document_sections_by_query_similarity("What are the best medicines for frequent colds with sneezing that increases in the morning?", document_embeddings)[:5]

[(0.8593520638413243, ('Asterias Rubens', 'Nose')),
 (0.8567244035263583, ('Scorpio', 'Nose')),
 (0.852248616978491, ('NymphÃ¦a Odorata', 'Respiratory Organs')),
 (0.8514100358805177, ('Lilium Tigrinum', 'Nose')),
 (0.8481693571963053, ('Stachys Betonica', 'Nose'))]

In [16]:
MAX_SECTION_LEN = 2000
SEPARATOR = "\n* "
ENCODING = "cl100k_base"  # encoding for text-davinci-003

encoding = tiktoken.get_encoding(ENCODING)
separator_len = len(encoding.encode(SEPARATOR))

f"Context separator contains {separator_len} tokens"

'Context separator contains 3 tokens'

In [17]:
def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame) -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index]
        
        chosen_sections_len += document_section.tokens + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + 'Medicine: ' + section_index[0] + ', Area: ' + section_index[1] + ', Symptoms: ' +
        document_section.content.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    # print(f"Selected {len(chosen_sections)} document sections:")
    # print("\n".join(chosen_sections_indexes))
    
    # header = """Answer the question as truthfully as possible using the provided context, and if the answer is not specified within the text below, say "I don't know."\n\nContext:\n"""
    header = """Provide medicines based on similarity of symptoms in question with those provided in context for the medicines. Give the best 5 medicines along with their symptoms.\n\nContext:\n"""
    # header = """\n\nContext:\n"""
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"

In [18]:
prompt = construct_prompt(
    "Medicine for frequent cold with sneezing upon waking up?",
    document_embeddings,
    df
)

print("===\n", prompt)

===
 Provide medicines based on similarity of symptoms in question with those provided in context for the medicines. Give the best 5 medicines along with their symptoms.

Context:

* Medicine: Scorpio, Area: Nose, Symptoms: Frequent sneezing.
* Medicine: Asterias Rubens, Area: Nose, Symptoms: Epistaxis; sneezing and coryza in the morning on waking.
* Medicine: Baryta Muriatica, Area: Nose, Symptoms: Sneezing in sleep. Coryza.
* Medicine: Plectranthus, Area: Nose, Symptoms: Excessive catarrh, frequent inclination to sneeze; great stoppage with at times discharge of tenacious yellow mucus. Dryness, mornings.
* Medicine: Lilium Tigrinum, Area: Nose, Symptoms: Sneezing and fluent coryza all day. Running from nose renewed after a glass of cold water; feeling of heat in face and over whole body as in fever; after a while prickling as in perspiration with only moist skin. Snuff has a greater effect than usual in evening.
* Medicine: Cornus Alternifolia, Area: Nose, Symptoms: Sneezing; head pa

In [19]:
def answer_query_with_context(
    query: str,
    df: pd.DataFrame,
    document_embeddings: dict[(str, str), np.array],
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt(
        query,
        document_embeddings,
        df
    )
    
    if show_prompt:
        print(prompt)

    response = openai.ChatCompletion.create(
    model=COMPLETIONS_MODEL,
    messages=[{'role': 'user', 'content': prompt}]
    )

    return response["choices"][0]["message"]["content"].strip(" \n")

In [20]:
answer_query_with_context("Medicine for frequent cold with sneezing upon waking up?", df, document_embeddings)

'1. Asterias Rubens - sneezing and coryza in the morning on waking.\n2. Baryta Muriatica - sneezing in sleep. Coryza.\n3. Lilium Tigrinum - sneezing and fluent coryza all day. Running from nose renewed after a glass of cold water; feeling of heat in face and over whole body as in fever; after a while prickling as in perspiration with only moist skin. Snuff has a greater effect than usual in evening.\n4. Hall - frequent (violent) sneezing and feeling of commencing coryza.\n5. Saponinum - dry coryza and frequent sneezing. R. nostril plugged in morning, with headache. Dull pain at root of nose and in temples. Nose cold and pale.'

In [21]:
answer_query_with_context("Medicine for headache at the top of head that feels better after applying a lot of pressuree?", df, document_embeddings)

'Symphytum and Mercurius Sulphuratus Ruber are two medicines that match this symptom. Other medicines that may be helpful include Helonias, Cinnabaris, and Diadema.'